In [3]:
import random
import collections

def count_values_classic(input_list,num_walkers):
    counter = collections.Counter(input_list)
    return sorted((val, counter[val]/num_walkers) for val in set(input_list))

def count_values_MQ(input_list,num_walkers):
    counter = collections.Counter(input_list)
    return sorted((val, counter[val]*counter[val]/(num_walkers)**3) for val in set(input_list))


def random_walk(num_steps=4, start=0, num_walkers=10000):
    """
    1-Dimensional Random Walk simulation.

    :param num_steps: The number of steps to simulate (default: 100)
    :param start: Starting position (default: 0)
    :return: A list containing the random walk's coordinates in the format [x, index].
    """

    # Initialize walk with starting position
    walk = [[ start for _ in range(num_walkers)]+[0]]

    for k in range(1, num_steps + 1):
        walk.append([k])
        for walker in range(num_walkers+1):
            # Append new position (x and index) to walk
            walk[-1].append(walk[-2][walker] + random.choice([-1, 1]))

    return walk

# Example usage:
num_walkers=10000
for step in random_walk(4,0,num_walkers):
    print(step[0],count_values_classic(step[1:],num_walkers))

for step in random_walk(4,0,num_walkers):
    print(step[0],count_values_MQ(step[1:],num_walkers))

0 [(0, 1.0)]
1 [(-1, 0.4979), (1, 0.5022)]
2 [(-2, 0.2469), (0, 0.496), (2, 0.2572)]
3 [(-3, 0.125), (-1, 0.3731), (1, 0.3722), (3, 0.1298)]
4 [(-4, 0.0633), (-2, 0.2471), (0, 0.3759), (2, 0.2512), (4, 0.0626)]
0 [(0, 11111111.111111112)]
1 [(-1, 2722500.0), (1, 2834733.4444444445)]
2 [(-2, 662596.0), (0, 2835856.0), (2, 698338.7777777778)]
3 [(-3, 176960.44444444444), (-1, 1517002.7777777778), (1, 1596853.4444444445), (3, 174445.44444444444)]
4 [(-4, 47089.0), (-2, 644273.7777777778), (0, 1588440.111111111), (2, 711211.1111111111), (4, 44240.11111111111)]
